# Multi-agent application

In [1]:
%pip install -U langgraph

   ---------------------------------------- 0.0/145.2 kB ? eta -:--:--
   -- ------------------------------------- 10.2/145.2 kB ? eta -:--:--
   -------- ------------------------------ 30.7/145.2 kB 435.7 kB/s eta 0:00:01
   ---------------------------------------  143.4/145.2 kB 1.4 MB/s eta 0:00:01
   ---------------------------------------- 145.2/145.2 kB 1.1 MB/s eta 0:00:00
  Attempting uninstall: langgraph-prebuilt
    Found existing installation: langgraph-prebuilt 0.1.7
    Uninstalling langgraph-prebuilt-0.1.7:
      Successfully uninstalled langgraph-prebuilt-0.1.7
  Attempting uninstall: langgraph
    Found existing installation: langgraph 0.3.20
    Uninstalling langgraph-0.3.20:
      Successfully uninstalled langgraph-0.3.20
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
from dotenv import load_dotenv
from langchain_openai import AzureOpenAI
from langchain_core.messages import HumanMessage
from langchain_openai import AzureChatOpenAI
import requests
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.tools import BaseTool, StructuredTool, tool

env_path = r"C:\Users\vaalt\OneDrive - Microsoft\Desktop\Projects - work\AI Agents in practice - Packt\Code\Chapter 7\.env"
load_dotenv(dotenv_path=env_path, override=True)

# Access the environment variables
openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
openai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_chat_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
tavily_api_key = os.getenv("TAVILY_API_KEY")
alphavantage_api_key = os.getenv("ALPHAVANTAGE_API_KEY")


In [2]:
# Initialize the Azure OpenAI model
llm = AzureChatOpenAI(
    openai_api_version=openai_api_version,
    azure_deployment=azure_chat_deployment,
)

# Create a human message and invoke the model
message = HumanMessage(
    content="Translate this sentence from English to French. I love programming."
)
response = llm.invoke([message])

# Print the response
print(response)

content="J'aime programmer." additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 19, 'total_tokens': 24, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_ee1d74bde0', 'id': 'chatcmpl-BQyHhXIcbuDJB0FYaplbtxoHlLr1N', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_code': {'filtered': False, 'dete

## Research Team

In [3]:
import json
import random
from datetime import datetime

# Sample list of stock symbols and sectors
assets = [
    {"symbol": "AAPL", "sector": "Technology"},
    {"symbol": "GOOGL", "sector": "Technology"},
    {"symbol": "MSFT", "sector": "Technology"},
    {"symbol": "AMZN", "sector": "Consumer Discretionary"},
    {"symbol": "TSLA", "sector": "Consumer Discretionary"},
    {"symbol": "JNJ", "sector": "Healthcare"},
    {"symbol": "NVDA", "sector": "Technology"},
    {"symbol": "XOM", "sector": "Energy"},
    {"symbol": "META", "sector": "Communication Services"},
    {"symbol": "V", "sector": "Financials"},
    {"symbol": "PG", "sector": "Consumer Staples"},
    {"symbol": "BABA", "sector": "Consumer Discretionary"},
    {"symbol": "JPM", "sector": "Financials"},
    {"symbol": "KO", "sector": "Consumer Staples"},
    {"symbol": "DIS", "sector": "Communication Services"},
    {"symbol": "PFE", "sector": "Healthcare"},
]

def generate_portfolio(assets):
    portfolio = []
    for asset in assets:
        quantity = random.randint(5, 100)
        purchase_price = round(random.uniform(50, 3000), 2)
        total_invested = round(quantity * purchase_price, 2)
        purchase_date = datetime.strftime(datetime(2022, random.randint(1,12), random.randint(1,28)), "%Y-%m-%d")

        portfolio.append({
            "symbol": asset["symbol"],
            "sector": asset["sector"],
            "quantity": quantity,
            "purchase_price": purchase_price,
            "total_invested": total_invested,
            "purchase_date": purchase_date
        })
    return portfolio

# Generate and save portfolio
portfolio_data = generate_portfolio(assets)

with open("sample_portfolio.json", "w") as f:
    json.dump(portfolio_data, f, indent=4)

print("✅ Portfolio JSON saved to 'sample_portfolio.json'")


✅ Portfolio JSON saved to 'sample_portfolio.json'


### Web tools

In [3]:
from typing import Annotated, List

from langchain_community.document_loaders import WebBaseLoader
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.tools import tool
# Load environment variables from .env file
load_dotenv()

tavily_tool = TavilySearchResults(max_results=5, tavily_api_key=tavily_api_key)


@tool
def scrape_webpages(urls: List[str]) -> str:
    """Use requests and bs4 to scrape the provided web pages for detailed information."""
    loader = WebBaseLoader(urls)
    docs = loader.load()
    return "\n\n".join(
        [
            f'<Document name="{doc.metadata.get("title", "")}">\n{doc.page_content}\n</Document>'
            for doc in docs
        ]
    )

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
from typing import List, Optional, Literal
from langchain_core.language_models.chat_models import BaseChatModel

from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.types import Command
from langchain_core.messages import HumanMessage, trim_messages


class State(MessagesState):
    next: str


def make_supervisor_node(llm: BaseChatModel, members: list[str]) -> str:
    options = ["FINISH"] + members
    system_prompt = (
        "You are a supervisor tasked with managing a conversation between the"
        f" following workers: {members}. Given the following user request,"
        " respond with the worker to act next. Each worker will perform a"
        " task and respond with their results and status. When finished,"
        " respond with FINISH."
    )

    class Router(TypedDict):
        """Worker to route to next. If no workers needed, route to FINISH."""

        next: Literal[*options]

    def supervisor_node(state: State) -> Command[Literal[*members, "__end__"]]:
        """An LLM-based router."""
        messages = [
            {"role": "system", "content": system_prompt},
        ] + state["messages"]
        response = llm.with_structured_output(Router).invoke(messages)
        goto = response["next"]
        if goto == "FINISH":
            goto = END

        return Command(goto=goto, update={"next": goto})

    return supervisor_node

In [5]:
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from typing_extensions import TypedDict

search_agent = create_react_agent(llm, tools=[tavily_tool])


def search_node(state: State) -> Command[Literal["supervisor"]]:
    result = search_agent.invoke(state)
    return Command(
        update={
            "messages": [
                HumanMessage(content=result["messages"][-1].content, name="search")
            ]
        },
        # We want our workers to ALWAYS "report back" to the supervisor when done
        goto="supervisor",
    )


web_scraper_agent = create_react_agent(llm, tools=[scrape_webpages])


def web_scraper_node(state: State) -> Command[Literal["supervisor"]]:
    result = web_scraper_agent.invoke(state)
    return Command(
        update={
            "messages": [
                HumanMessage(content=result["messages"][-1].content, name="web_scraper")
            ]
        },
        # We want our workers to ALWAYS "report back" to the supervisor when done
        goto="supervisor",
    )




research_supervisor_node = make_supervisor_node(llm, ["search", "web_scraper"])

In [6]:
research_builder = StateGraph(State)
research_builder.add_node("supervisor", research_supervisor_node)
research_builder.add_node("search", search_node)
research_builder.add_node("web_scraper", web_scraper_node)
#research_builder.add_node("alpha_vantage_agent", alpha_vantage_node)

research_builder.add_edge(START, "supervisor")
research_graph = research_builder.compile()

In [9]:
for s in research_graph.stream(
    {"messages": [("user", "what's the price of apple stock today and some recent news about it?")]},
    {"recursion_limit": 100},
):
    print(s)
    print("---")

{'supervisor': {'next': 'search'}}
---
{'search': {'messages': [HumanMessage(content='### Apple Stock Price Today\nThe current stock price of Apple Inc. (AAPL) is approximately **$209.28** as of the market close. During today\'s trading, the stock ranged from **$206.20 to $209.75**. \n\nFor more details, you can visit [WSJ - Apple Inc. (AAPL) Stock Price](https://www.wsj.com/market-data/quotes/AAPL) or [Apple Investor Relations](https://investor.apple.com/stock-price/default.aspx).\n\n---\n\n### Recent News About Apple Stock\n1. **Earnings Preview:** Apple is set to report its fiscal Q2 2025 earnings on **May 1, after market close**. Wall Street analysts expect earnings per share (EPS) of **$1.61**, up from **$1.53** year-over-year. Revenue is forecasted to reach **$94.07 billion**, higher than the prior year\'s $90.75 billion. [Source: Yahoo Finance](https://finance.yahoo.com/quote/AAPL/news/)\n\n2. **Analyst Target Downward Revision:** Bank of America recently lowered its stock price

# Portfolio analysis Team

### Portfolio read tool

In [9]:
import json

@tool
def read_sample_portfolio(json_path: str = "sample_portfolio.json") -> str:
    """
    Reads the sample_portfolio.json file and returns its content as a string.
    Each entry includes the stock symbol, sector, quantity, purchase price, and purchase date.
    """
    if not os.path.exists(json_path):
        return f"File not found: {json_path}"

    with open(json_path, "r") as f:
        portfolio = json.load(f)

    if not isinstance(portfolio, list):
        return "Unexpected portfolio format."

    response = "Sample Portfolio:\n"
    for stock in portfolio:
        response += (
            f"- {stock['symbol']} ({stock['sector']}): "
            f"{stock['quantity']} shares @ ${stock['purchase_price']} "
            f"(Bought on {stock['purchase_date']})\n"
        )
    return response

### Portfolio analyzer tool

In [10]:
from langchain.tools import tool
from langchain_core.language_models import BaseLanguageModel
from typing import List, Dict

@tool
def analyze_portfolio(
    portfolio: List[str]
) -> str:
    """
    Generate a professional summary of the investment portfolio using the provided LLM.
    """
    llm = AzureChatOpenAI(
        openai_api_version=openai_api_version,
        azure_deployment=azure_chat_deployment,
    )

    # Construct a prompt for the LLM
    prompt = (
        "You are a financial analyst. Based on the following investment portfolio data, "
        "provide a concise and professional summary highlighting diversification, sector exposure, "
        "and any notable observations:\n\n"
        f"{portfolio}\n\n"
        "Summary:"
    )

    # Generate the summary using the LLM
    try:
        response = llm.invoke(prompt)
        return response
    except Exception as e:
        return f"An error occurred while generating the summary: {e}"


In [11]:
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from typing_extensions import TypedDict


analyze_portfolio_agent = create_react_agent(llm, tools=[analyze_portfolio])


def analyze_portfolio_node(state: State) -> Command[Literal["supervisor"]]:
    result = analyze_portfolio_agent.invoke(state)
    return Command(
        update={
            "messages": [
                HumanMessage(content=result["messages"][-1].content, name="analyze_portfolio")
            ]
        },
        # We want our workers to ALWAYS "report back" to the supervisor when done
        goto="supervisor",
    )

read_portfolio_agent = create_react_agent(llm, tools=[read_sample_portfolio])


def read_portfolio_node(state: State) -> Command[Literal["supervisor"]]:
    result = read_portfolio_agent.invoke(state)
    return Command(
        update={
            "messages": [
                HumanMessage(content=result["messages"][-1].content, name="read_portfolio")
            ]
        },
        # We want our workers to ALWAYS "report back" to the supervisor when done
        goto="supervisor",
    )


portfolio_supervisor_node = make_supervisor_node(llm, [ "read_portfolio", "analyze_portfolio"])

In [12]:
portfolio_builder = StateGraph(State)
portfolio_builder.add_node("supervisor", portfolio_supervisor_node)
portfolio_builder.add_node("analyze_portfolio", analyze_portfolio_node)
portfolio_builder.add_node("read_portfolio", read_portfolio_node)


portfolio_builder.add_edge(START, "supervisor")
portfolio_graph = portfolio_builder.compile()

In [13]:
for s in portfolio_graph.stream(
    {"messages": [("user", "evaluate my portfolio.")]},
    {"recursion_limit": 100},
):
    print(s)
    print("---")

{'supervisor': {'next': 'read_portfolio'}}
---
{'read_portfolio': {'messages': [HumanMessage(content="Your portfolio is diverse, spanning multiple sectors. Here's a breakdown of its composition and potential evaluation:\n\n### Portfolio Details:\n\n#### Summary By Sector\n- **Technology**:\n  - Apple (AAPL): 13 shares\n  - Google (GOOGL): 21 shares\n  - Microsoft (MSFT): 68 shares\n  - NVIDIA (NVDA): 72 shares\n\n- **Consumer Discretionary**:\n  - Amazon (AMZN): 86 shares\n  - Tesla (TSLA): 15 shares\n  - Alibaba (BABA): 49 shares\n\n- **Healthcare**:\n  - Johnson & Johnson (JNJ): 67 shares\n  - Pfizer (PFE): 54 shares\n\n- **Energy**:\n  - Exxon Mobil (XOM): 93 shares\n\n- **Communication Services**:\n  - Meta (META): 100 shares\n  - Disney (DIS): 29 shares\n\n- **Financials**:\n  - Visa (V): 93 shares\n  - JPMorgan (JPM): 10 shares\n\n- **Consumer Staples**:\n  - Procter & Gamble (PG): 7 shares\n  - Coca-Cola (KO): 95 shares\n\n### Current Market Prices & Performance Evaluation:\nFor

# Report generator team

In [14]:
from pathlib import Path
from tempfile import TemporaryDirectory
from typing import Dict, Optional
from typing_extensions import TypedDict

# Define a real directory path
REAL_DIRECTORY = Path(r"C:\Users\vaalt\OneDrive - Microsoft\Desktop\Projects - work\AI Agents in practice - Packt\Code\Chapter 7\outputs")

#_TEMP_DIRECTORY = TemporaryDirectory()
WORKING_DIRECTORY = Path(REAL_DIRECTORY)

In [ ]:

@tool
def read_document(
    file_name: Annotated[str, "File path to read the document from."],
    start: Annotated[Optional[int], "The start line. Default is 0"] = None,
    end: Annotated[Optional[int], "The end line. Default is None"] = None,
) -> str:
    """Read the specified document."""
    with (WORKING_DIRECTORY / file_name).open("r") as file:
        lines = file.readlines()
    if start is None:
        start = 0
    return "\n".join(lines[start:end])


@tool
def write_document(
    content: Annotated[str, "Text content to be written into the document."],
    file_name: Annotated[str, "File path to save the document."],
) -> Annotated[str, "Path of the saved document file."]:
    """Create and save a text document."""
    with (WORKING_DIRECTORY / file_name).open("w") as file:
        file.write(content)
    return f"Document saved to {file_name}"


@tool
def edit_document(
    file_name: Annotated[str, "Path of the document to be edited."],
    inserts: Annotated[
        Dict[int, str],
        "Dictionary where key is the line number (1-indexed) and value is the text to be inserted at that line.",
    ],
) -> Annotated[str, "Path of the edited document file."]:
    """Edit a document by inserting text at specific line numbers."""

    with (WORKING_DIRECTORY / file_name).open("r") as file:
        lines = file.readlines()

    sorted_inserts = sorted(inserts.items())

    for line_number, text in sorted_inserts:
        if 1 <= line_number <= len(lines) + 1:
            lines.insert(line_number - 1, text + "\n")
        else:
            return f"Error: Line number {line_number} is out of range."

    with (WORKING_DIRECTORY / file_name).open("w") as file:
        file.writelines(lines)

    return f"Document edited and saved to {file_name}"

In [ ]:
doc_writer_agent = create_react_agent(
    llm,
    tools=[write_document, edit_document, read_document],
    prompt=(
        "You can read, write and edit documents. "
        "Your job is to write a well-structured document."
        "You can also edit existing documents. "
        "Once the document is finished, you can save it."
        "Don't ask follow-up questions."
    ),
)


def doc_writing_node(state: State) -> Command[Literal["supervisor"]]:
    result = doc_writer_agent.invoke(state)
    return Command(
        update={
            "messages": [
                HumanMessage(content=result["messages"][-1].content, name="doc_writer")
            ]
        },
        # We want our workers to ALWAYS "report back" to the supervisor when done
        goto="supervisor",
    )


reviewer_agent = create_react_agent(
    llm,
    tools=[read_document, write_document],
    prompt=(
        "You can read documents and review the content. "
        "You can modify the content with approapriate improvements."
        "Don't ask follow-up questions."
    ),
)


def reviewer_node(state: State) -> Command[Literal["supervisor"]]:
    result = reviewer_agent.invoke(state)
    return Command(
        update={
            "messages": [
                HumanMessage(content=result["messages"][-1].content, name="reviewer")
            ]
        },
        # We want our workers to ALWAYS "report back" to the supervisor when done
        goto="supervisor",
    )





doc_writing_supervisor_node = make_supervisor_node(
    llm, ["doc_writer", "reviewer"]
)

In [ ]:
# Create the graph here
paper_writing_builder = StateGraph(State)
paper_writing_builder.add_node("supervisor", doc_writing_supervisor_node)
paper_writing_builder.add_node("doc_writer", doc_writing_node)
paper_writing_builder.add_node("reviewer", reviewer_node)

paper_writing_builder.add_edge(START, "supervisor")
paper_writing_graph = paper_writing_builder.compile()

In [28]:
for s in paper_writing_graph.stream(
    {
        "messages": [
            (
                "user",
                "Write a short poem about cats and dogs",
            )
        ]
    },
    {"recursion_limit": 100},
):
    print(s)
    print("---")

{'supervisor': {'next': 'doc_writer'}}
---
{'doc_writer': {'messages': [HumanMessage(content="Here’s a short poem about cats and dogs:\n\n---\n\nIn the sunlit yard, the dogs do leap,  \nBounding high, their joy runs deep.  \nA wagging tail, a bark so sweet,  \nTheir zest for life is hard to beat.  \n\nOn windowsills, the cats recline,  \nWith purring grace, their worlds align.  \nA gentle stare, a silky paw,  \nThey carry themselves with timeless awe.  \n\nBoth so different, yet hearts so pure,  \nLove and laughter, their gifts endure.  \nCats and dogs, a charming pair,  \nFilling our lives with moments rare.  \n\n---\n\nLet me know if you'd like this poem saved as a document.", additional_kwargs={}, response_metadata={}, name='doc_writer', id='7e91bd79-09c6-46fd-b411-ea135967bd29')]}}
---
{'supervisor': {'next': 'note_taker'}}
---
{'note_taker': {'messages': [HumanMessage(content='The poem has been saved as "cats_and_dogs_poem.txt". Let me know if there\'s anything else you\'d like!',

# All together

In [ ]:
from langchain_core.messages import BaseMessage

teams_supervisor_node = make_supervisor_node(llm, ["research_team","portfolio_team", "writing_team"])


In [30]:
def call_research_team(state: State) -> Command[Literal["supervisor"]]:
    response = research_graph.invoke({"messages": state["messages"][-1]})
    return Command(
        update={
            "messages": [
                HumanMessage(
                    content=response["messages"][-1].content, name="research_team"
                )
            ]
        },
        goto="supervisor",
    )

def call_portfolio_team(state: State) -> Command[Literal["supervisor"]]:
    response = portfolio_graph.invoke({"messages": state["messages"][-1]})
    return Command(
        update={
            "messages": [
                HumanMessage(
                    content=response["messages"][-1].content, name="portfolio_team"
                )
            ]
        },
        goto="supervisor",
    )

def call_paper_writing_team(state: State) -> Command[Literal["supervisor"]]:
    response = paper_writing_graph.invoke({"messages": state["messages"][-1]})
    return Command(
        update={
            "messages": [
                HumanMessage(
                    content=response["messages"][-1].content, name="writing_team"
                )
            ]
        },
        goto="supervisor",
    )


# Define the graph.
super_builder = StateGraph(State)
super_builder.add_node("supervisor", teams_supervisor_node)
super_builder.add_node("research_team", call_research_team)
super_builder.add_node("portfolio_team", call_portfolio_team)
super_builder.add_node("writing_team", call_paper_writing_team)

super_builder.add_edge(START, "supervisor")
super_graph = super_builder.compile()

In [32]:
for s in super_graph.stream(
    {
        "messages": [
            ("user", "Generate a well structured report on how to improve my portfolio given the current market situation.")
        ],
    },
    {"recursion_limit": 150},
):
    print(s)
    print("---")

{'supervisor': {'next': 'research_team'}}
---
{'research_team': {'messages': [HumanMessage(content="Certainly! Below is a refined and well-structured report that provides actionable steps to enhance your portfolio in light of the current market dynamics.\n\n---\n\n### **Portfolio Improvement Report: Navigating the Current Market (October 2023)**\n\n---\n\n#### **1. Current Market Landscape**\nTo make effective portfolio adjustments, it is essential to understand the existing market context. Based on recent data:\n\n- **Equities**: Global equities have retraced gains from earlier in the year, facing heightened volatility (-3% in October but +8.6% YTD).\n- **Interest Rates**: The Federal Reserve maintains a hawkish outlook amid persistent inflation, contributing to elevated Treasury yields (10-year at ~4.93%).\n- **Commodities and Precious Metals**: Energy prices have softened despite geopolitical tensions, while gold has gained traction as a safe-haven asset.\n- **Real Estate**: Housing